<a href="https://colab.research.google.com/github/RicNavarro/data-cleaning-nashville/blob/main/DC_Rebirth_List.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [346]:
import numpy as np
import pandas as pd
import glob as gb
import os
import re

In [347]:
df_main = pd.read_csv("/content/DC_Rebirth_Comic_Order.csv")

In [348]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [349]:
df_main.head()

,Comic Name
0,DC Universe: Rebirth #1 (2016)
1,The Flash: Rebirth Vol. 2 #1 (2016)
2,Nightwing: Rebirth #1 (2016)
3,Titans: Rebirth #1 (2016)
4,Titans Vol. 3 #1 (2016)


Formatação inicial do texto para retirar espaços, parênteses, numerações sobre a edição e demais informações que não são relevantes para o propósito de nossa análise

In [350]:
def extract_and_clean_name(name):
    if name.split(" ")[0] == 'Read':
        return 'EVENT'
    else:
        return (name.title().split(':')[0].split('Vol')[0].split('#')[0]).rstrip().split("Annual")[-1].split("Special")[-1].upper()

df_main['Character'] = df_main['Comic Name'].apply(extract_and_clean_name)

In [351]:
df_main

,Comic Name,Character
0,DC Universe: Rebirth #1 (2016),DC UNIVERSE
1,The Flash: Rebirth Vol. 2 #1 (2016),THE FLASH
2,Nightwing: Rebirth #1 (2016),NIGHTWING
3,Titans: Rebirth #1 (2016),TITANS
4,Titans Vol. 3 #1 (2016),TITANS
...,...,...
2479,Read Dark Nights: Death Metal here.,EVENT
2480,Generations Shattered #1 (2021),GENERATIONS SHATTERED
2481,Generations Forged #1 (2021),GENERATIONS FORGED
2482,Read Endless Winter here.,EVENT


**Eventização**

O código abaixo deve ser retrabalhado para que todas as entradas onde há 'Character' igual a 'EVENT' sejam stripadas de 'Read ' e ' here.'

In [352]:
#print("Read Night of the Monster Men.".strip('Read ').strip( here.))

def clean_event(x):
  return x.replace('Read ', '').replace(' here.', '')

events_true = df_main['Character'] == "EVENT"
print(events_true)

df_main.loc[events_true, 'Comic Name'] = df_main.loc[events_true, 'Comic Name'].apply(clean_event)

0       False
1       False
2       False
3       False
4       False
        ...  
2479     True
2480    False
2481    False
2482     True
2483     True
Name: Character, Length: 2484, dtype: bool


In [353]:
df_main['Event'] = np.where(df_main['Character'] == "EVENT", df_main['Comic Name'], "None")
df_main['Event']

,Event
0,None
1,None
2,None
3,None
4,None
...,...
2479,Dark Nights: Death Metal
2480,None
2481,None
2482,Endless Winter


In [354]:
df_main

,Comic Name,Character,Event
0,DC Universe: Rebirth #1 (2016),DC UNIVERSE,None
1,The Flash: Rebirth Vol. 2 #1 (2016),THE FLASH,None
2,Nightwing: Rebirth #1 (2016),NIGHTWING,None
3,Titans: Rebirth #1 (2016),TITANS,None
4,Titans Vol. 3 #1 (2016),TITANS,None
...,...,...,...
2479,Dark Nights: Death Metal,EVENT,Dark Nights: Death Metal
2480,Generations Shattered #1 (2021),GENERATIONS SHATTERED,None
2481,Generations Forged #1 (2021),GENERATIONS FORGED,None
2482,Endless Winter,EVENT,Endless Winter


Vamos ter de usar expressões regulares para poder modificar adequadamente as entradas na Série `events_list`, removendo espaços e pontuações para que nossa comparação com o nome dos arquivos seja adequada.

Consta no código uma explicação sobre as expressões

In [355]:
events_list = df_main.loc[df_main['Event'] != 'None']['Event']

def reformat_name_event(event):
    # Substituir qualquer caractere n alfanum (exceto _) ou espaço por um underscore
    #    Regex: [^\w\s] -> qualquer coisa que NÃO seja (caractere alfanumérico OU espaço)
    #    | -> OU
    #    \s+ -> um ou mais espaços
    event = re.sub(r'[^\w\s]|\s+', '_', event)

    # Remover underscores duplicados que possam surgir (ex: "Night__of___the")
    event = re.sub(r'__+', '_', event)

    # Remover underscores no início ou fim da string
    event = event.strip('_')

    return event


events_list = events_list.apply(reformat_name_event)

Temos que ler os nomes dos arquivos em DC_Events e

**(A)** Conforme fazemos a leitura armazenamos na lista `df_events`, que depois é reordenada segundo `events_dict` ou similares

ou

**(B)** Fazemos a leitura já checando os elementos de `events_dict`, ordenando conforme lemos e armazenando em `dfs_events` já na ordem

In [358]:
events_path = "/content/DC_Events/"
fileformat = "*.csv"
fileformat_noast = ".csv"
event_names = []
dfs_events = []

path_events = gb.glob(f"{events_path}/{fileformat}")

#for i in events_list.values:
#  print(i)
for i in path_events:
  print(os.path.basename(i))

if (len(path_events) == len(events_list.values)):
  for i in range(len(path_events)):
    #print(events_path + events_list.values[i] + fileformat_noast)
    try:
      event_address = events_path + events_list.values[i] + fileformat_noast
      dfs_events.append(pd.read_csv(event_address))
    except:
      #print(event_address)
      print("O evento " + events_list.values[i] + " não tem aquivo de nome equivalente")

teste1 = pd.read_csv("/content/DC_Events/Future_State.csv")
teste1.head()

#print(os.path.basename(path_events))
#    print(os.path.basename(i).replace(".csv",""))

#for i in path_events:
#  print(i)
#  event_names.append(os.path.basename(i))
#  dfs_events.append(pd.read_csv(i))
#print(event_names)

The_Infected.csv
Heroes_in_Crisis.csv
Justice_League_vs_Suicide_Squad.csv
The_Lazarus_Contract.csv
Superman_Reborn.csv
Drowned_Earth.csv
Dark_Nights_Death_Metal.csv
City_of_Bane.csv
Joker_War.csv
Event_Leviathan.csv
Endless_Winter.csv
Future_State.csv
Night_of_the_Monster_Men.csv
Dark_Nights_Metal.csv
O evento Future_State não tem aquivo de nome equivalente


ParserError: Error tokenizing data. C error: Expected 1 fields in line 37, saw 2


In [357]:
print(event_names[i])

Dark_Knights_Death_Metal
Dark_Nights_Death_Metal

Future_State
Future_State

IndexError: list index out of range